In [0]:
%run ../../Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
%sql
SELECT * from customers;

In [0]:
%sql DESCRIBE customers;

In [0]:
%sql
SELECT customer_id, profile:first_name, profile:address:country
FROM customers;

In [0]:
%sql
SELECT from_json(profile) AS profile_struct
FROM customers;

In [0]:
%sql
SELECT profile from customers LIMIT 1;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tv_parsed_customers AS SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Amble","last_name":"Pedrick","gender":"Male","address":{"street":"02354 Holmberg Trail","city":"Petřvald","country":"Czech Republic"}}')) AS profile_struct
FROM customers;

select * from tv_parsed_customers;

In [0]:
%sql DESCRIBE tv_parsed_customers

In [0]:
%sql
SELECT customer_id, profile_struct.first_name, profile_struct.address.country
FROM tv_parsed_customers;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tv_customers_finals AS
  SELECT customer_id, profile_struct.*
  FROM tv_parsed_customers;

  SELECT * from tv_customers_finals;

In [0]:
%sql
select order_id, customer_id, books from orders;


In [0]:
%sql
select order_id, customer_id, explode(books) from orders;


In [0]:
%sql
select customer_id
, collect_set(order_id)
, collect_set(books.book_id) 
from orders 
group by customer_id;


In [0]:
%sql
select customer_id
, collect_set(books.book_id) as before_flatten
, array_distinct(flatten(collect_set(books.book_id))) as after_flatten
from orders 
group by customer_id;


In [0]:
%sql
create or replace view orders_enriched as
select *
from (
  select *, explode(books) as book
  from orders) o
inner join books b
on o.book.book_id = b.book_id;

select * from orders_enriched;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tv_orders_updates
AS select * from parquet.`/Volumes/datademo/default/bookstore_dataset/orders-new`;

select * from orders 
union 
select * from tv_orders_updates;

In [0]:
%sql
select * from orders
intersect
select * from tv_orders_updates;

In [0]:
%sql
select * from orders
minus
select * from tv_orders_updates;

In [0]:
%sql
CREATE OR REPLACE TABLE transactions AS

SELECT * FROM (
  SELECT
    customer_id,
    book.book_id AS book_id,
    book.quantity AS quantity
  FROM orders_enriched
) PIVOT (
  sum(quantity) FOR book_id in (
    'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
    'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
  )
);

SELECT * FROM transactions